Read an available Data Set into a Pandas Dataframe

In [4]:
# Main data packages. 
import numpy as np
import pandas as pd

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter


import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

Customize Seaborn and Pandas Plots

In [5]:
sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': '.9', 'grid.color': '.8'}
)
cmaps_hex = ['#193251','#FF5A36','#1E4485', '#99D04A','#FF5A36', '#DB6668']
sns.set_palette(palette=cmaps_hex)
sns_c = sns.color_palette(palette=cmaps_hex)
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

Take a first look at the "imbalance_de.csv" file:

In [6]:
# Import data
raw_df = pd.read_csv('data/imbalance_de.csv')
raw_df.head()

,2013-12-31 23:00:00,707.5,156.36
0,2013-12-31 23:15:00,966.296,81.23
1,2013-12-31 23:30:00,1018.984,104.01
2,2013-12-31 23:45:00,867.940,60.51
3,2014-01-01 00:00:00,1738.004,239.03
4,2014-01-01 00:15:00,1436.952,89.82


In [7]:
# Import data
es_cross_border_flow_df = pd.read_csv('data/es_crossboarder_flows.csv')
es_cross_border_flow_df.head()

,dt_start_utc,power_mw,from_country,to_country,type
0,2016-12-31 23:00:00,636,DE,DK,crossboarder_flow
1,2016-12-31 23:00:00,2331,DE,NL,crossboarder_flow
2,2016-12-31 23:00:00,1,DE,SE,crossboarder_flow
3,2016-12-31 23:00:00,585,DK,DE,crossboarder_flow
4,2016-12-31 23:00:00,0,NL,DE,crossboarder_flow


Try to enumerate the numbers of prognoses per "date_prognosticated" from the data set "EQ_epex_da_prognosis.csv" and use the last (most recent) one of them (tip from Jonas)

Give the columns fitting names:

In [8]:
eq_epex_da_prognosis_df = pd.read_csv("data/EQ_epex_da_prognosis.csv")

In [9]:
eq_epex_da_prognosis_df.columns = ["date_prognosticated", "prognosis", "date_issued"]

In [10]:
eq_epex_da_prognosis_df.head()

,date_prognosticated,prognosis,date_issued
0,2018-12-28 00:00:00,35.95,2018-12-26 23:00:00
1,2018-12-28 01:00:00,34.18,2018-12-26 23:00:00
2,2018-12-28 02:00:00,34.00,2018-12-26 23:00:00
3,2018-12-28 03:00:00,34.79,2018-12-26 23:00:00
4,2018-12-28 04:00:00,37.88,2018-12-26 23:00:00


In [11]:
len(eq_epex_da_prognosis_df["date_prognosticated"])

483096

Put all unique prognosticated dates from EQ_epex_da_prognosis.csv" in a list called "unique_dates_list" for later use:

In [12]:
unique_dates_list = eq_epex_da_prognosis_df["date_prognosticated"].unique().tolist()

In [13]:
len(unique_dates_list)

23014

In [14]:
def prognosis_count(df):
    """
    takes a given DataFrame as parameter and creates an extra column
    "prognosis_count", where the numbers of
    prognoses per unique date are counted
    """
    # Add empty column to eq_epex_da_prognosis_df called "prognosis_count" 
    # to count the numbers of prognoses for a given time stamp:
    df["prognosis_count"] = np.nan
    # In a for loop, the amounts of prognoses per unique date are counted
    # and saved in th prognosis_count column
    j = 1
    df["prognosis_count"][0]= int(1)
    for i in range (1, len(df["date_prognosticated"])):
        if df["date_prognosticated"][i] == df["date_prognosticated"][i-1]:
            j += 1
            df["prognosis_count"][i] = int(j)
        else:
            j = 1
            df["prognosis_count"][i] = int(j)
    # transforms the values in prognosis_count to integers
    df["prognosis_count"] = df["prognosis_count"].values.astype(int)
    # resets the index
    df.reset_index()
    # saves the df to a csv file in the data/data_processed folder 
    df.to_csv("data/data_processed/EQ_epex_da_prognosis_count.csv", index=False)
    
    return df

In [15]:
prognosis_count(eq_epex_da_prognosis_df)

,date_prognosticated,prognosis,date_issued,prognosis_count
0,2018-12-28 00:00:00,35.95,2018-12-26 23:00:00,1
1,2018-12-28 01:00:00,34.18,2018-12-26 23:00:00,1
2,2018-12-28 02:00:00,34.00,2018-12-26 23:00:00,1
3,2018-12-28 03:00:00,34.79,2018-12-26 23:00:00,1
4,2018-12-28 04:00:00,37.88,2018-12-26 23:00:00,1
...,...,...,...,...
483091,2021-08-12 20:00:00,88.28,2021-07-01 22:00:00,4
483092,2021-08-12 21:00:00,78.91,2021-06-28 22:00:00,1
483093,2021-08-12 21:00:00,78.32,2021-06-29 22:00:00,2
483094,2021-08-12 21:00:00,79.96,2021-06-30 22:00:00,3


In [17]:
eq_epex_da_prognosis_df.loc[7,"prognosis"]

55.39

Write a function that transforms the Data Frame with the prognosis_count column (eq_epex_da_prognosis_df) to a Data Frame "eq_epex_da_last_prognosis_df" which only contains the prognosticated time stamp and the last prognosis for this timestamp:

In [18]:
# import json
def current_prognosis(Input_df):
    """
    This function takes the "eq_epex_da_prognosis_df" as parameter "Input_df"
    and gets rid of all rows with older prognoses
    """
    # Create a dictionary of nested dictionaries which contain the number of 
    # prognosis_counts as keys and the prognoses as value for each unique 
    # date_prognosticated as key:
    dict = Input_df.groupby("date_prognosticated")[["prognosis_count", "prognosis"]] \
      .apply(lambda x: x.set_index("prognosis_count").to_dict(orient="index")) \
      .to_dict()

    # initialize empty Data Frame and fill it with input from nested dict:
    eq_epex_da_last_prognosis_df = pd.DataFrame(columns=["timestamp", "prognosis"])
    
    # iterate through the nested dict keys and append the "timestamp" as key and
    # the last_prognosis as "prognosis":
    for key in dict:
        last_prognosis = list(dict[key].values())[-1]
        eq_epex_da_last_prognosis_df = eq_epex_da_last_prognosis_df.append({'timestamp': key,
                                                                            'prognosis': last_prognosis["prognosis"],
                                                                           }, ignore_index=True)
    # save the resulting DataFrame into a csv file:
    eq_epex_da_last_prognosis_df.to_csv("data/data_processed/EQ_epex_da_last_prognosis_count.csv", index=False)
    
    return eq_epex_da_last_prognosis_df
    
    
    

In [19]:
current_prognosis(eq_epex_da_prognosis_df)

,timestamp,prognosis
0,2018-12-28 00:00:00,35.95
1,2018-12-28 01:00:00,34.18
2,2018-12-28 02:00:00,34.00
3,2018-12-28 03:00:00,34.79
4,2018-12-28 04:00:00,37.88
...,...,...
23009,2021-08-12 17:00:00,98.67
23010,2021-08-12 18:00:00,97.19
23011,2021-08-12 19:00:00,92.55
23012,2021-08-12 20:00:00,88.28


# regelleistung_aggr_results.csv: Data processing

Now, work on the regelleistung_aggr_results.csv and transform it in a way that there are only unique timestamps, with all the additional timestamps transformed to columns in the same row:

In [20]:
# Import data
regelleistung_aggr_results_df = pd.read_csv('data/regelleistung_aggr_results.csv')
#regelleistung_aggr_results_df.head()

Get rid of the date_end column, as it contains the same values like the date_start column:

In [21]:
regelleistung_aggr_results_df.drop(["date_end"], axis=1, inplace=True)

Add a column to the Data Frame "regelleistung_aggr_results_df" which is the combination of the columns "product" and "reserve_type" and the column shall be named "product_reserve_type". This will be our unique key.

In [22]:
regelleistung_aggr_results_df.eval("product_reserve_type= product + reserve_type", inplace=True)

In [23]:
# drop unnecessary columns:
regelleistung_aggr_results_df.drop(["product", "reserve_type"], axis=1, inplace=True)

# Caution!

In order to make below defined function work, it is necessary to put all data files from Jonas into a separate subfolder called "data", which in itself contains a sub-folder called "data_processed" in order to make the csv saving process defined in the function (close to the end) work!

In [98]:
def regelleistung_aggr_results_transform(regelleistung_df, file_name):
    """
    This function takes in a modified (see above) "regelleistung_df" plus a string-name for the
    csv-datfile which saves the end result as input parameters
    and writes all multiple rows who have the same time stamp into a 
    new DataFrame stored as the new csv file), where
    all those row cells are written into newly named columns in one row per unique time stamp
    """
    # build a list of the unique entries in the "product_reserve_type" column:
    product_reserve_type_list = regelleistung_df["product_reserve_type"].unique().tolist()
    #print(product_reserve_type_list)
    # create a list of all column names from the csv file "regelleistung_aggr_results.csv":
    unique_columns = regelleistung_df.columns.tolist()
    
    # take all the unique time stamps form "regelleistung_aggr_results.csv" and create a list:
    unique_dates = regelleistung_df["date_start"].unique().tolist()
    
    # construct a (mostly empty) array(df) from the unique date_starts and with columns with a prefix from the
    # column "product_reserve_type" plus the unique_columns from the basic list(columns 1 to 13),
    # then fill the cells of the new columns with 0s
    
    repeat_col_number = regelleistung_df.shape[1] - 2
    
    df_test = pd.DataFrame(unique_dates,columns=["date_start"])  
    for product_reserve_type in product_reserve_type_list:
        for column in unique_columns[1:repeat_col_number+1]:
            df_test[f"{product_reserve_type}_{column}"] = 0.00
    print(df_test.shape, len(regelleistung_df["product_reserve_type"]) - 1)
    
    # create a list of all column names from "df_test":
    df_test_columns = df_test.columns.unique().values.tolist()
    
    # take the information from the cells from the additional same timestamp rows and
    # write it into the corresponding cells of "df_test" to transform the multiline
    # data content of the original csv into one row in the dataframe "df_test":

    row_enum_input, row_enum_output, prt_list_enum, m = 0, 0, 0, 0
    col_enum_input, col_enum_output = 1, 1
    
    # calculate parameters which are specific to the input_df to improve reusability of function:
    
    max_row_input_df = len(regelleistung_df["product_reserve_type"]) - 1
    max_row_output_df = len(df_test["date_start"]) - 1
    max_col_enum_output_df = df_test.shape[1] - 1
    
    max_repeats = len(product_reserve_type_list) - 1
    print(f"max_row_input_df:{max_row_input_df}\nmax_row_output_df:{max_row_output_df}\nmax_col_enum_output_df:{max_col_enum_output_df}")
    print(f"repeat_col_number: {repeat_col_number}\nmax_repeats: {max_repeats}")
    for row_enum_input in range(max_row_input_df + 1):
        col_sec_it = 1
        l = 0
        while col_enum_output <= max_col_enum_output_df + 1:
            if row_enum_input == max_row_input_df or row_enum_output > max_row_output_df :
                break
            elif regelleistung_df["product_reserve_type"][row_enum_input] == product_reserve_type_list[prt_list_enum] and regelleistung_df["date_start"][row_enum_input] == df_test["date_start"][row_enum_output] and col_enum_output < max_col_enum_output_df:
                #print(f"Test normal_route start = row_enum_input: {row_enum_input},  col_enum_input: {col_enum_input}, col_sec_it: {col_sec_it} l: {l},  prt_list_enum: {prt_list_enum}, row_enum_output: {row_enum_output}, col_enum_output: {col_enum_output}")
                for l in range (col_sec_it, col_sec_it + repeat_col_number):  
                    df_test[df_test_columns[l]][row_enum_output] = regelleistung_df[unique_columns[col_enum_input]][row_enum_input]
                    col_enum_input += 1
                    col_enum_output += 1
                if m < max_repeats:
                    m += 1
                    prt_list_enum += 1 
                    col_enum_input = 1
                    col_sec_it = 1 + m * repeat_col_number
                    if row_enum_input < len(regelleistung_df["product_reserve_type"]) - 1:
                        row_enum_input += 1
                #print(f"Test normal: m:{m},l: {l}, col_enum_input: {col_enum_input}, row_enum_input: {row_enum_input}, col_sec_it: {col_sec_it}, col_enum_output: {col_enum_output}, prt_list_enum: {prt_list_enum}, row_enum_input: {row_enum_input}")
            elif regelleistung_df["product_reserve_type"][row_enum_input] != product_reserve_type_list[prt_list_enum] and regelleistung_df["date_start"][row_enum_input] == df_test["date_start"][row_enum_output] and col_enum_output < max_col_enum_output_df:
                if m < max_repeats:
                    col_enum_output += repeat_col_number
                    prt_list_enum += 1
                    m += 1
                    col_sec_it = 1 + m * repeat_col_number
                col_enum_input = 1
                #print(f"Test elif route =, row_enum_input: {row_enum_input}, l: {l}, col_sec_it: {col_sec_it}, prt_list_enum: {prt_list_enum}")
            else:
                col_sec_it = 1
                col_enum_output = 1
                col_enum_input = 1
                prt_list_enum = 0
                row_enum_input += 1
                row_enum_output += 1
                m = 0
                l = 0             
                #print(f"Test else_route =, l: {l} , row_enum_output: {row_enum_output}, col_sec_it: {col_sec_it}, prt_list_enum: {prt_list_enum}, col_enum_output: {col_enum_output}")            
    print(prt_list_enum, col_sec_it, row_enum_input, row_enum_output) 
    df_test.to_csv(f'data/data_processed/{file_name}.csv', index=False)
    
    return df_test.head(5)
    

In [96]:
regelleistung_aggr_results_df["product_reserve_type"][1]

'NEG_00_04SRL'

In [99]:
regelleistung_aggr_results_transform(regelleistung_aggr_results_df, "regelleistung_aggr_results")

(674, 313) 16067
max_row_input_df:16067
max_row_output_df:673
max_col_enum_output_df:312
repeat_col_number: 13
max_repeats: 23
0 1 16067 674


,date_start,NEG_00_04MRL_total_min_capacity_price_eur_mw,NEG_00_04MRL_total_average_capacity_price_eur_mw,NEG_00_04MRL_total_marginal_capacity_price_eur_mw,NEG_00_04MRL_total_min_energy_price_eur_mwh,NEG_00_04MRL_total_average_energy_price_eur_mwh,NEG_00_04MRL_total_marginal_energy_price_eur_mwh,NEG_00_04MRL_germany_min_capacity_price_eur_mw,NEG_00_04MRL_germany_average_capacity_price_eur_mw,NEG_00_04MRL_germany_marginal_capacity_price_eur_mw,...,POS_20_24SRL_total_min_energy_price_eur_mwh,POS_20_24SRL_total_average_energy_price_eur_mwh,POS_20_24SRL_total_marginal_energy_price_eur_mwh,POS_20_24SRL_germany_min_capacity_price_eur_mw,POS_20_24SRL_germany_average_capacity_price_eur_mw,POS_20_24SRL_germany_marginal_capacity_price_eur_mw,POS_20_24SRL_germany_min_energy_price_eur_mwh,POS_20_24SRL_germany_average_energy_price_eur_mwh,POS_20_24SRL_germany_marginal_energy_price_eur_mwh,POS_20_24SRL_germany_import_export_mw
0,2019-01-01,0.0,150.81,872.262,1.0,-371.64,-18422.0,0.0,150.81,872.262,...,37.10,75.60,106.20,0.00,12.05,22.30,37.10,75.60,106.20,0.0
1,2019-01-02,0.0,130.94,292.500,5.0,-163.22,-1250.0,0.0,130.94,292.500,...,55.97,84.74,106.25,0.00,9.62,18.10,55.97,84.74,106.25,0.0
2,2019-01-03,0.0,9.60,30.026,15.0,-145.65,-600.0,0.0,9.60,30.026,...,62.00,87.53,106.25,0.00,8.13,15.48,62.00,87.53,106.25,0.0
3,2019-01-04,0.0,5.08,9.900,21.0,-37.13,-277.0,0.0,5.08,9.900,...,68.00,80.52,97.00,3.12,9.31,12.14,68.00,80.52,97.00,0.0
4,2019-01-05,0.0,4.96,17.323,18.0,-220.96,-600.0,0.0,4.96,17.323,...,64.67,80.29,101.20,2.40,8.71,13.00,64.67,80.29,101.20,0.0


# regelleistung_demand.csv: Data Processing

Now, work on the regelleistung_demand.csv and transform it in a way that there are only unique timestamps, with all the additional timestamps transformed to columns in the same row:

In [100]:
# Import data
regelleistung_demand_df = pd.read_csv('data/regelleistung_demand.csv')
# Get rid of "date_end" column:
regelleistung_demand_df.drop(["date_end"], axis=1, inplace=True)

Add a column to the Data Frame "regelleistung_demand_df" which is the combination of the columns "product" and "reserve_type" and the column shall be named "product_reserve_type". This will be our unique key for our nested_dict.

In [101]:
regelleistung_demand_df.eval("product_reserve_type= product + reserve_type", inplace=True)

In [102]:
# drop unnecessary columns:
regelleistung_demand_df.drop(["product", "reserve_type"], axis=1, inplace=True)

In [103]:
regelleistung_demand_df.shape

(16188, 4)

In [104]:
regelleistung_aggr_results_transform(regelleistung_demand_df, "regelleistung_demand_processed")

(681, 49) 16187
max_row_input_df:16187
max_row_output_df:680
max_col_enum_output_df:48
repeat_col_number: 2
max_repeats: 23
0 1 16187 681


,date_start,NEG_00_04MRL_total_demand_mw,NEG_00_04MRL_germany_block_demand_mw,NEG_00_04SRL_total_demand_mw,NEG_00_04SRL_germany_block_demand_mw,NEG_04_08MRL_total_demand_mw,NEG_04_08MRL_germany_block_demand_mw,NEG_04_08SRL_total_demand_mw,NEG_04_08SRL_germany_block_demand_mw,NEG_08_12MRL_total_demand_mw,...,POS_12_16SRL_total_demand_mw,POS_12_16SRL_germany_block_demand_mw,POS_16_20MRL_total_demand_mw,POS_16_20MRL_germany_block_demand_mw,POS_16_20SRL_total_demand_mw,POS_16_20SRL_germany_block_demand_mw,POS_20_24MRL_total_demand_mw,POS_20_24MRL_germany_block_demand_mw,POS_20_24SRL_total_demand_mw,POS_20_24SRL_germany_block_demand_mw
0,2019-01-01,1067.0,1067.0,1760.0,1760.0,1067.0,1067.0,1760.0,1760.0,1067.0,...,1882.0,1882.0,874.0,874.0,1882.0,1882.0,874.0,874.0,1882.0,1882.0
1,2019-01-02,1067.0,1067.0,1760.0,1760.0,1067.0,1067.0,1760.0,1760.0,1067.0,...,1882.0,1882.0,874.0,874.0,1882.0,1882.0,874.0,874.0,1882.0,1882.0
2,2019-01-03,1067.0,1067.0,1760.0,1760.0,1067.0,1067.0,1760.0,1760.0,1067.0,...,1882.0,1882.0,874.0,874.0,1882.0,1882.0,874.0,874.0,1882.0,1882.0
3,2019-01-04,1067.0,1067.0,1760.0,1760.0,1067.0,1067.0,1760.0,1760.0,1067.0,...,1882.0,1882.0,874.0,874.0,1882.0,1882.0,874.0,874.0,1882.0,1882.0
4,2019-01-05,1067.0,1067.0,1760.0,1760.0,1067.0,1067.0,1760.0,1760.0,1067.0,...,1882.0,1882.0,874.0,874.0,1882.0,1882.0,874.0,874.0,1882.0,1882.0


In [105]:
dict_regelleistung = regelleistung_aggr_results_df.groupby("date_start") \
      .apply(lambda x: x.set_index("product_reserve_type").to_dict(orient="index"))

In [183]:
#pd.get_dummies(regelleistung_aggr_results_df, columns = ["product_reserve_type"],  prefix="", prefix_sep='')

List of all relevant columns in regelleistung_aggr_results_df:

List of all relevant columns in df_test_columns: